In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
df = pd.read_csv("../input/pp-merge/nodups_train.csv")
display(df)


In [ ]:
print(df['labels'].value_counts())

In [ ]:
# # label split

label2id = {
    'complex' : 0, 
    'frog_eye_leaf_spot': 1, 
    'powdery_mildew' : 2, 
    'rust' : 3 , 
    'scab' : 4,
    'healthy' : 5
}

display(df.head())
df["labels"] = df["labels"].map(lambda x : [i for i in x.split(" ")])
display(df.head())
df["labels"] = df["labels"].map(lambda x : [label2id[i] for i in x])
display(df.head())

In [ ]:
# # label 하나씩으로 나누기
df_new = pd.DataFrame()

for image_data, label_data in df.values:
    if len(label_data) > 1 :
        for label in label_data:
            tmp_list= [image_data, label]
            df_new=df_new.append(pd.Series(tmp_list, index= df.columns), ignore_index=True)
    else:
        tmp_list= [image_data, label_data[0]]
        df_new=df_new.append(pd.Series(tmp_list, index= df.columns), ignore_index=True)
        
df_new = df_new.astype({'labels':'int32'})         
display(df_new)            

# df_new.to_csv('../input/pp-merge/one_label.csv', index=False)

In [ ]:
# label 하나씩인 df
# df_new = pd.read_csv('../input/pp-merge/one_label.csv', index=False)

display(df_new) 
display(df_new['labels'].value_counts())

In [ ]:
# label별로 균일하게 data split
sfk = StratifiedKFold(5)
for train_idx, valid_idx in sfk.split(df_new['image'], df_new['labels']):
    df_train = df_new.iloc[train_idx]
    df_valid = df_new.iloc[valid_idx]
    break
    
print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

# df_train.to_csv('./data/plant-pathology-2021-fgvc8/merge/train_data.csv', index=False)
# df_valid.to_csv('./data/plant-pathology-2021-fgvc8/merge/validation_data.csv', index=False)

In [ ]:
df_train['labels'].unique()

In [ ]:
# # Imagegenerator

# train_datagen = ImageDataGenerator(rescale= 1/255)
# validation_datagen = ImageDataGenerator(rescale= 1/255)

# df_train = df_train.astype({'labels':'str'})
# df_valid = df_valid.astype({'labels':'str'})

# train_generator = train_datagen.flow_from_dataframe(
#     df_train,
#     directory = '../input/plant-pathology-2021-fgvc8/train_images/',
#     x_col = 'image',
#     y_col = 'labels',
#     target_size = (150,150),
#     color_mode = 'rgb',
#     class_mode = 'sparse',
#     batch_size = 20
# )

# validation_generator = validation_datagen.flow_from_dataframe(
#     df_valid,
#     directory = '../input/plant-pathology-2021-fgvc8/train_images/',
#     x_col = 'image',
#     y_col = 'labels',
#     target_size = (150,150),
#     color_mode = 'rgb',
#     class_mode = 'sparse',
#     batch_size = 20
# )


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale= 1/255)
df_train = df_train.astype({'labels':'str'})
df_valid = df_valid.astype({'labels':'str'})
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    directory = '../input/plant-pathology-2021-fgvc8/train_images',
    x_col = 'image',
    y_col = 'labels',
    target_size = (150,150),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 20
)
validation_generator = validation_datagen.flow_from_dataframe(
    df_valid,
    directory = '../input/plant-pathology-2021-fgvc8/train_images',
    x_col = 'image',
    y_col = 'labels',
    target_size = (150,150),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 20
)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


model = Sequential()

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu',
                 
                 input_shape=(150,150,3)))

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=512,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=512,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=512,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))


model.add(Flatten())
model.add(Dense(units=512,
                activation='relu'))

model.add(Dropout(rate=0.5))

model.add(Dense(units=128,
                activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(units=6,
                activation='softmax'))

print(model.summary())


In [ ]:
METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.2,
        patience=3,
        verbose=1
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

In [ ]:
# # First HP
# from tensorflow.keras.optimizers import RMSprop,Adam
# import tensorflow_addons as tfa

# epochs = 30
# batch_size = 50
# optimizer = Adam(lr = 0.0001)
# model.compile(optimizer = optimizer,
#              loss = 'CategoricalCrossentropy',
#              metrics = ['accuracy'])
# callbacks = create_callbacks()
# history = model.fit(train_generator,
#                     epochs = epochs,
#                     steps_per_epoch=100,
#                     validation_data = validation_generator,
#                     verbose=1, 
#                     callbacks = callbacks)

In [ ]:
# model.save('../input/plant-pathology-2021-fgvc8/cnn.h5')

In [ ]:
# figure, axis = plt.subplots(2, 1, figsize=(15,15))
# axis.ravel()
# axis[0].plot(history.history['accuracy'],label='Training Data')
# axis[0].plot(history.history['val_accuracy'], label='Validation Data')
# axis[0].set(xlabel='Epochs',ylabel='Accuracy', title='Accuracy vs Epochs')
# axis[0].legend(loc="upper left")

# axis[1].plot(history.history['loss'], label='Training Data')
# axis[1].plot(history.history['val_loss'], label='Validation Data')
# axis[1].set(xlabel='Epochs',ylabel='Loss', title='Categorical Crossentropy Loss vs Epochs')
# axis[1].legend(loc="upper left")

# plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
smote_train = pd.read_csv("../input/pp-merge/train_data.csv")
display(smote_train)

In [ ]:
# # label별로 균일하게 data split
# sfk = StratifiedKFold(5)
# for train_idx, valid_idx in sfk.split(df_new['image'], df_new['labels']):
#     df_train = df_new.iloc[train_idx]
#     df_valid = df_new.iloc[valid_idx]
#     break
    
# print(f"train size: {len(df_train)}")
# print(f"valid size: {len(df_valid)}")

# # df_train.to_csv('./data/plant-pathology-2021-fgvc8/merge/train_data.csv', index=False)
# # df_valid.to_csv('./data/plant-pathology-2021-fgvc8/merge/validation_data.csv', index=False)

In [ ]:
y_train = smote_train['labels'].values

In [ ]:
x_index = list(smote_train.columns.values)[1:]

In [ ]:
x_train = smote_train[x_index].values

In [ ]:
smote_valid = pd.read_csv("../input/pp-merge/validation_data.csv")

In [ ]:
y_valid = smote_valid['labels'].values

In [ ]:
x_val_index = list(smote_valid.columns.values)[1:]

In [ ]:
x_valid=smote_valid[x_val_index].values

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


model = Sequential()

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu',
                 
                 input_shape=(28,28,3)))

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(filters=128,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=256,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=512,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))

model.add(Conv2D(filters=512,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))


model.add(Flatten())
model.add(Dense(units=512,
                activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(units=128,
                activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(units=6,
                activation='softmax'))

print(model.summary())


In [ ]:
x_train

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])


history = model.fit(x_train.reshape(-1,28,28,3),
                    y_train,
                    steps_per_epoch=250,
                    epochs=50,
                    batch_size=100,
                    verbose=1
                    )

In [ ]:
model.save('../kaggle/working/smote_cnn.h5')

In [ ]:
test_df = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
test_dir = '../input/plant-pathology-2021-fgvc8/test_images/'
# test_df = pd.DataFrame()
# test_df['image'] = os.listdir(test_dir)

In [ ]:
test

In [ ]:
datagen = ImageDataGenerator(rescale=1/255)

test_generator = datagen.flow_from_dataframe(
    test_df,
    directory = '../input/plant-pathology-2021-fgvc8/test_images',
    x_col = 'image',
    y_col = 'labels',
    target_size = (28,28),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = 20
)

In [ ]:
prediction = model.predict(test_generator)
print(prediction)

In [ ]:
test_label_list = []
for i in prediction:
    test_result = np.argmax(i)
    
    test_label_list.append(test_result)

In [ ]:
labels_new_dict = { 'complex' : 0, 
                    'frog_eye_leaf_spot': 1, 
                    'powdery_mildew' : 2, 
                    'rust' : 3 , 
                    'scab' : 4,
                    'healthy' : 5}

for key, value in labels_new_dict.items():
    for i in range(len(test_label_list)):
        if test_label_list[i] == value:
            test_label_list[i] = key
print(test_label_list) 

In [ ]:
print(test_label_list)

In [ ]:
test_image_list = os.listdir('../input/plant-pathology-2021-fgvc8/test_images') 
final_df = pd.DataFrame()
final_df['image'] = test_df['image']
final_df['labels'] = test_label_list
display(final_df)
# test_df.to_csv('/home/lab09/data/plant-pathology-2021-fgvc8/test_result.csv', index=False)
final_df.to_csv('./submission.csv', index=False)

In [ ]:
# test_df.to_csv('/home/lab09/data/plant-pathology-2021-fgvc8/test_result.csv', index=False)
test_df.to_csv('./submission.csv', index=False)

In [ ]:
# First HP
from tensorflow.keras.optimizers import RMSprop,Adam
import tensorflow_addons as tfa

epochs = 30
batch_size = 50
optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer,
             loss = 'SparseCategoricalCrossentropy',
             metrics = ['SparseCategoricalCrossentropy'])
callbacks = create_callbacks()
history = model.fit(train_generator,
                    epochs = epochs,
                    steps_per_epoch=200,
                    validation_data = validation_generator,
                    verbose=1, 
                    callbacks = callbacks)